# 概要
[台東区来訪者アンケート調査結果](https://www.city.taito.lg.jp/kusei/online/opendata/iryo/raihousha_anke-to.html)の自由記述回答をテキスト分析してみる．

よくあるアンケートの形式は主な項目が選択肢として用意され，それらのいずれにも該当しない回答がある場合に備えて「その他(具体的に書いてください)」的な自由記述欄が用意されている．

この自由記述欄の分析で得られる情報:
- 新しく選択肢として追加した方が良い項目
- アンケート実施者が気づかなかった項目



# 準備

## データの準備
台東区のサイトからデータをダウンロードして，文字コードを変換する．

In [ ]:
%%bash
# nkfは文字コード変換用のコマンド
apt-get install nkf
wget "https://www.city.taito.lg.jp/kusei/online/opendata/iryo/raihousha_anke-to.files/03raihousha-kekka.csv"
# Shift-jisなのでutf-8に変換する
nkf -w 03raihousha-kekka.csv > 03raihousha-kekka.utf8.csv

wget "https://www.city.taito.lg.jp/kusei/online/opendata/iryo/raihousha_anke-to.files/02raihousha-code.csv"
# Shift-jisなのでutf-8に変換する
nkf -w 02raihousha-code.csv > 02raihousha-code.utf8.csv


## いつもの準備

In [ ]:
%%bash
apt-get install mecab libmecab-dev mecab-ipadic-utf8
pip install mecab-python3 ipadic
ln -s /etc/mecabrc /usr/local/etc/mecabrc
pip install nlplot
pip install japanize-matplotlib

In [ ]:
import MeCab
import re
import nlplot
mecab=MeCab.Tagger()

import pandas as pd
import csv

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
sns.set()
%matplotlib inline
import japanize_matplotlib

from collections import defaultdict
import math

In [ ]:
# 回答のコードと内容の対応
codes=defaultdict(lambda:defaultdict())
with open('02raihousha-code.utf8.csv') as fh:
  cr=csv.reader(fh)
  header=next(cr)
  for row in cr:
    for i in range(2,len(row)):
      if row[i]=='':
        break
      line=re.split('\:',row[i])
      codes[row[1]][int(line[0])]=line[1]


In [ ]:
def mecabparse(sentence):
  result=[]
  morphs=re.split('\n',mecab.parse(sentence).rstrip())
  for m in morphs:
    if m=='EOS':
      break
    line=re.split('\t',m)
    features=re.split(',',line[1])
    if re.match('名詞|動詞|形容詞|副詞',features[0]) and not re.match('非自立|代名詞',features[1]):
      if features[6]=='*':
        result.append(line[0])
      else:
        result.append(features[6])
  return result

In [ ]:
questionnaire=pd.read_csv("03raihousha-kekka.utf8.csv")

In [ ]:
questionnaire.head()

# 「問11．現在の観光地および台東区への旅行に対する満足度についてお聞かせ下さい」の理由の分析
- 現在の観光地に関しては「問⑧_調査地点」にあるので，それ毎に分ける



## 前処理
- 理由が空欄のものは削除する
- 調査地点の回答をコードから地名に置換する
- 理由の単語列の取得

In [ ]:
df=questionnaire.dropna(subset=['問⑪_総合満足度理由'])
df=df[['問⑧_調査地点','問⑪_⑧総合満足度','問⑪_総合満足度理由']]
df['問⑧_調査地点']=df['問⑧_調査地点'].apply(lambda x:codes['問⑧_調査地点'][x])
df['words']=df['問⑪_総合満足度理由'].apply(mecabparse)

In [ ]:
df.head()

In [ ]:
df['問⑧_調査地点'].value_counts() # 度数分布

## 各観光地毎の満足度の分布を表示

In [ ]:
for c in codes['問⑧_調査地点'].values():
  print(c)
  print(df[df['問⑧_調査地点']==c]['問⑪_⑧総合満足度'].value_counts())

概ね，好意的な意見が多そうなので満足度別での表示はしない．

## 各調査地点毎のワードクラウドを表示してみる．

In [ ]:
for c in codes['問⑧_調査地点'].values():
  print(c)
  subdf=df[df['問⑧_調査地点']==c]
  npt=nlplot.NLPlot(subdf,target_col='words')
  wc=npt.wordcloud(
    max_words=200,
    max_font_size=100,
    colormap='tab20_r'
  )
  plt.figure(figsize=(8, 6))
  plt.imshow(wc, interpolation="bilinear")
  plt.axis("off")
  plt.show()

## 調査地点毎にtfidfを表示してみる
調査地点毎に理由をまとめて一つのテキストと見なしてtfidfを計算する

In [ ]:
# tfidfの計算
wordfreq=defaultdict(lambda:defaultdict(int))
num_of_words=defaultdict(int)
docf=defaultdict(int)
for c in codes['問⑧_調査地点'].values():
  subdf=df[df['問⑧_調査地点']==c]
  for bow in subdf['words']:
    for w in bow:
      wordfreq[c][w]+=1
      num_of_words[c]+=1
  for w in wordfreq[c]:
    docf[w]+=1



In [ ]:
tfidf=defaultdict(lambda:defaultdict(float))
for c in wordfreq:
  for w in wordfreq[c]:
    tfidf[c][w]=(wordfreq[c][w]/num_of_words[c])*(math.log(len(wordfreq)/docf[w]))
  print(c)
  srted=sorted(tfidf[c].keys(),key=lambda x:tfidf[c][x],reverse=True)
  for i in range(10):
    print('',srted[i],tfidf[c][srted[i]],sep='\t')


# 課題
台東区来訪者アンケート調査結果の「問 16．最後に、台東区の観光行政に関するご意見を自由に記入してください。」の回答を分析しなさい．
- 何をどう分析するかは自由
  - どんな意見が多かったか，など
- アンケートの他の項目を利用してもよい
  - 例: 回答者の属性毎にどういう意見があるかを整理する
- 意見にはいくつかタイプが見られそう
  - 「〜してほしい」といった要望
  - 「〜が良かった」といった感想
